In [3]:
"""
Shows SpamAssassin Python Wrapper at work.

https://pypi.org/project/spamassassin-client/


- SETUP: -
1) Have spamasassin installed
    > sudo apt get spamassassin
2) > pip install spamassassin_client
3) Start spamassassin server
    > sudo spamd
4) Use spamassassin_client

- NOTE: -
REMEMBER TO SHUT DOWN SPAMD SERVER WHEN DONE

"""
import os
from spamassassin_client import SpamAssassin


In [4]:
ex_spam = 'data/sample-spam.txt'
ex_ham = 'data/sample-nonspam.txt'
files = [ex_spam, ex_ham]

for fp in files:
    with open(fp, 'rb') as f: # need to read bytes
        print(f"File: {fp}")
        assassin = SpamAssassin(f.read())
        spam_score = assassin.get_score()
        if assassin.is_spam():
            print(f"DETECTED SPAM: {spam_score}")
            print(assassin.get_fulltext())
        else:
            print(f"NOT SPAM: {spam_score}")
            print(assassin.get_fulltext())
    print("----")

File: data/sample-spam.txt
DETECTED SPAM: 1000.0
Spam detection software, running on the system "Nicks-MBP-8.lan",
has identified this incoming email as possible spam.  The original
message has been attached to this so you can view it or label
similar future email.  If you have any questions, see
the administrator of that system for details.

Content preview:  This is the GTUBE, the Generic Test for Unsolicited Bulk Email
If your spam filter supports it, the GTUBE provides a test by which you can
verify that the filter is installed correctly and is detecting incoming spam.
You can send yourself a test mail containing t [...]

Content analysis details:   (1000.0 points, 5.0 required)

pts rule name              description
---- ---------------------- --------------------------------------------------
-0.0 NO_RECEIVED            Informational: message has no Received headers
-0.0 NO_RELAYS              Informational: message was not relayed via SMTP
1000 GTUBE                  BODY: Gene